In [6]:
# Initialize BigQuery Client (assuming this is already done and authenticated)
try:
    client = bigquery.Client()
except Exception as e:
    print(f"Error initializing BigQuery client. Ensure authentication is set up. Error: {e}")
    # Exit or handle if client initialization fails
    exit()


DATASET_ID = 'bigquery-public-data.thelook_ecommerce'

def get_dataset_schema(dataset_id):
    """
    Lists all tables in the specified dataset and prints their schema details.
    """
    print(f"--- Inspecting Tables in Dataset: {dataset_id} ---\n")

    try:
        # 1. List all tables in the dataset
        tables = list(client.list_tables(dataset_id))

        if not tables:
            print(f"No tables found in dataset {dataset_id}.")
            return

        for table_item in tables:
            table_id = f"{table_item.project}.{table_item.dataset_id}.{table_item.table_id}"
            table_name = table_item.table_id

            print(f"\n=======================================================")
            print(f"TABLE NAME: {table_name}")
            print(f"=======================================================")

            # 2. Get the detailed schema for the table
            table = client.get_table(table_id)

            schema_info = []
            for field in table.schema:
                schema_info.append({
                    'Column Name': field.name,
                    'Data Type': field.field_type,
                    'Required/Nullable': field.mode,
                    'Description': field.description
                })

            # 3. Display schema as a clean DataFrame
            df_schema = pd.DataFrame(schema_info)

            # Use markdown format for easy copying/pasting
            print(df_schema.to_markdown(index=False))
            print("\n")

    except Exception as e:
        print(f"An error occurred while accessing the dataset: {e}")

# Run the inspection function
get_dataset_schema(DATASET_ID)

--- Inspecting Tables in Dataset: bigquery-public-data.thelook_ecommerce ---


TABLE NAME: distribution_centers
| Column Name              | Data Type   | Required/Nullable   | Description   |
|:-------------------------|:------------|:--------------------|:--------------|
| id                       | INTEGER     | NULLABLE            |               |
| name                     | STRING      | NULLABLE            |               |
| latitude                 | FLOAT       | NULLABLE            |               |
| longitude                | FLOAT       | NULLABLE            |               |
| distribution_center_geom | GEOGRAPHY   | NULLABLE            |               |



TABLE NAME: events
| Column Name     | Data Type   | Required/Nullable   | Description   |
|:----------------|:------------|:--------------------|:--------------|
| id              | INTEGER     | NULLABLE            |               |
| user_id         | INTEGER     | NULLABLE            |               |
| sequence_

## **DISCOVER:**

### Prompt One: Initial DB Access



> This project will use the following public data set from big query: bigquery-public-data.thelook_ecommerce. Using BigQuery SQL access the dataset and present a head.



# Task
Access the `bigquery-public-data.thelook_ecommerce.orders` table using BigQuery SQL and present its head.

## Access BigQuery Dataset

### Subtask:
Using BigQuery SQL, I will access the `bigquery-public-data.thelook_ecommerce.orders` table, select all columns, and limit the query to the first 10 rows. I will then load the results into a pandas DataFrame.


**Reasoning**:
I will now write the Python code to connect to BigQuery, execute the specified SQL query to fetch the first 10 rows from the `orders` table, and load the results into a pandas DataFrame named `df_orders` as per the instructions.



In [1]:
from google.cloud import bigquery

# Construct a BigQuery client object.
client = bigquery.Client()

sql = """
    SELECT *
    FROM `bigquery-public-data.thelook_ecommerce.orders`
    LIMIT 10
"""

df_orders = client.query(sql).to_dataframe()

df_orders.head()

,order_id,user_id,status,gender,created_at,returned_at,shipped_at,delivered_at,num_of_item
0,22,22,Cancelled,F,2022-04-12 03:14:00+00:00,NaT,NaT,NaT,4
1,34,30,Cancelled,F,2025-07-02 02:58:00+00:00,NaT,NaT,NaT,2
2,38,33,Cancelled,F,2019-12-28 18:20:00+00:00,NaT,NaT,NaT,1
3,40,33,Cancelled,F,2019-11-21 18:20:00+00:00,NaT,NaT,NaT,1
4,54,40,Cancelled,F,2024-10-29 08:14:00+00:00,NaT,NaT,NaT,1


### Prompt 2: KPI Generation


> Identify the top 3 growth KPIs for the business (e.g., 90-day revenue trend, repeat purchase rate, average order value).



### Top 3 E-commerce Growth KPIs:

### One: Average Order Value (AOV)

**Definition:** The average dollar amount spent each time a customer places an order.

Why it's a Growth KPI: Increasing AOV is a powerful lever for growth because it boosts total revenue without requiring additional marketing spend to acquire new customers. Growth strategies focus on cross-selling, upselling, and optimizing minimum spend thresholds.
Data Required	Source Table	Sample Column Used
Total Revenue	order_items	sale_price (or similar price column)
Total Successful Orders	orders	order_id (Filtered by status = 'Complete')


### Two: Repeat Purchase Rate (RPR)

**Definition:** The percentage of customers who have made more than one purchase within a defined time frame (e.g., last 12 months).

Why it's a Growth KPI: Retaining existing customers is significantly cheaper than acquiring new ones. A high RPR indicates strong customer satisfaction, brand loyalty, and healthy Customer Lifetime Value (CLV). Growing RPR is a core indicator of sustainable growth.
Data Required	Source Table	Sample Column Used
Customer Identifier	orders	user_id
Order Date	orders	created_at


### Three: Net Order Count Growth (Month-over-Month or Quarter-over-Quarter)

**Definition:** The rate of change in the number of successful orders placed over a specific time period compared to the previous period.

Why it's a Growth KPI: This metric measures the raw expansion and scaling of the business's operations and market penetration. It reflects the immediate success of marketing and acquisition efforts in driving transaction volume.
Data Required	Source Table	Sample Column Used
Order Identifier	orders	order_id
Order Status	orders	status (Must filter for 'Complete' or 'Shipped')
Order Date	orders	created_at


###Prompt 3: KPI Trends

> Use CTEs and window functions to compute trends and MoM/YoY growth for Net Order Count Growth. Use python to write BigQuery SQL

In [2]:
import pandas as pd
import os

# Set up BigQuery Client
# Assuming application-default credentials are set up or running in a Google Cloud environment
try:
    client = bigquery.Client()
except Exception as e:
    print(f"Error initializing BigQuery client. Ensure authentication is set up. Error: {e}")
    # Handle environment where BQ client cannot be initialized gracefully

# --- BigQuery SQL Query using CTEs and Window Functions ---
QUERY = """
WITH
    -- CTE 1: Aggregate successful order counts by month
    monthly_orders AS (
        SELECT
            -- Extract Year and Month for proper temporal sorting
            FORMAT_DATE('%Y-%m', DATE(created_at)) AS order_month,
            COUNT(order_id) AS order_count
        FROM
            `bigquery-public-data.thelook_ecommerce.orders`
        WHERE
            -- Filter for successful orders (Net Order Count)
            status IN ('Complete', 'Shipped')
        GROUP BY 1
        ORDER BY 1
    ),

    -- CTE 2: Apply Window Functions (LAG) to fetch previous periods
    calculate_growth AS (
        SELECT
            order_month,
            order_count,

            -- Retrieve the order count from the immediately preceding month (MoM lag = 1)
            LAG(order_count, 1) OVER (ORDER BY order_month) AS previous_month_count,

            -- Retrieve the order count from the same month 1 year prior (YoY lag = 12)
            LAG(order_count, 12) OVER (ORDER BY order_month) AS previous_year_count
        FROM
            monthly_orders
    )

-- Final SELECT: Calculate the MoM and YoY growth rates
SELECT
    order_month,
    order_count,

    -- Calculate MoM Growth Rate: (Current - Previous Month) / Previous Month
    -- SAFE_DIVIDE handles cases where the denominator might be zero or null (e.g., first month)
    SAFE_DIVIDE(
        (order_count - previous_month_count),
        previous_month_count
    ) AS mom_growth_rate,

    -- Calculate YoY Growth Rate: (Current - Previous Year) / Previous Year
    SAFE_DIVIDE(
        (order_count - previous_year_count),
        previous_year_count
    ) AS yoy_growth_rate
FROM
    calculate_growth
WHERE
    -- Filter out initial months where YoY growth cannot be calculated (first 12 months)
    previous_year_count IS NOT NULL
ORDER BY
    order_month DESC
"""

# Execute the query
print("Executing BigQuery query to calculate MoM and YoY Net Order Count Growth...")
try:
    # Run the query and convert to a Pandas DataFrame
    net_order_growth_df = client.query(QUERY).to_dataframe()

    # --- Presentation ---

    # Format the growth rates for better readability (percentage)
    pd.options.display.float_format = '{:,.2%}'.format

    print("\n--- Net Order Count Growth (MoM & YoY) Trends ---")
    print("Net Order Count: Successful orders (status IN 'Complete', 'Shipped')")

    # Display the most recent trends
    print(net_order_growth_df.head(15))

except NameError:
    print("Cannot run query: BigQuery client was not successfully initialized.")
except Exception as e:
    print(f"An error occurred during query execution: {e}")


Executing BigQuery query to calculate MoM and YoY Net Order Count Growth...

--- Net Order Count Growth (MoM & YoY) Trends ---
Net Order Count: Successful orders (status IN 'Complete', 'Shipped')
   order_month  order_count  mom_growth_rate  yoy_growth_rate
0      2025-10         4280           20.94%          157.37%
1      2025-09         3539           13.68%          142.23%
2      2025-08         3113           10.31%          124.12%
3      2025-07         2822           13.29%           97.48%
4      2025-06         2491            5.24%           93.70%
5      2025-05         2367            8.63%           76.25%
6      2025-04         2179            6.40%           80.98%
7      2025-03         2048           14.99%           68.56%
8      2025-02         1781           -3.89%           66.60%
9      2025-01         1853            7.42%           64.71%
10     2024-12         1725            6.61%           56.11%
11     2024-11         1618           -2.71%           66.12

## **INESTIGATE:**

### Prompt 4: Deep Dive

> Lets choose one product catergory and one customer segment. Use SQL to explore drivers (discounts, marketing channel if available, region, device). Assume all libraries and dependecies set in previous steps are in the notebook. Use Python.

This analysis will focus on understanding the purchasing behavior drivers for a valuable segment—Repeat Customers—buying a popular product category, which we will assume to be Jeans.

Target Selection:

    Product Category: Jeans (Assumed popular category)
    Customer Segment: Repeat Customers (Users with more than 1 completed order)

SQL Strategy:

    CTE 1 (repeat_users): Identify the user_ids belonging to the Repeat Customer segment.
    CTE 2 (target_transactions): Join the necessary tables (orders, order_items, products, users). Filter this data set to include only transactions by the repeat_users for the category Jeans.
    Final Query: Group the results by the key drivers (device, country, discount_tier) to calculate AOV and total revenue for each combination.



In [8]:
import pandas as pd
from google.cloud import bigquery
import os

# Initialize BigQuery Client (assuming this is already done and authenticated)
try:
    client = bigquery.Client()
except Exception as e:
    print(f"Error initializing BigQuery client: {e}")

# Define the category and segment parameters
TARGET_CATEGORY = 'Jeans'
MIN_ORDERS_FOR_REPEAT = 2

# --- CORRECTED BigQuery SQL Query (Replacing 'device' with 'traffic_source') ---
QUERY_DRIVERS_CORRECTED = f"""
WITH
    -- CTE 1: Identify Repeat Customers (Users with 2 or more successful orders)
    repeat_users AS (
        SELECT
            user_id
        FROM
            `bigquery-public-data.thelook_ecommerce.orders`
        WHERE
            status IN ('Complete', 'Shipped')
        GROUP BY
            user_id
        HAVING
            COUNT(order_id) >= {MIN_ORDERS_FOR_REPEAT}
    ),

    -- CTE 2: Filter transactions for the target segment and category
    target_transactions AS (
        SELECT
            t1.order_id,
            t1.user_id,
            t2.sale_price,
            t3.retail_price,
            t3.category,
            -- CORRECTION: Replacing non-existent t1.device with t4.traffic_source
            t4.traffic_source,
            t4.country,

            -- Calculate the discount amount (Product Retail Price - Sale Price)
            (t3.retail_price - t2.sale_price) AS discount_amount,

            -- Calculate discount percentage relative to the original retail price
            SAFE_DIVIDE(
                (t3.retail_price - t2.sale_price),
                t3.retail_price
            ) AS discount_pct,

            -- Create Discount Tiers for grouping
            CASE
                -- Using discount_pct for tiering. Check if discount is 0 first.
                WHEN (t3.retail_price - t2.sale_price) <= 0 THEN '0_No_Discount'
                WHEN SAFE_DIVIDE((t3.retail_price - t2.sale_price), t3.retail_price) < 0.10 THEN '1_Low_Discount_1_10'
                WHEN SAFE_DIVIDE((t3.retail_price - t2.sale_price), t3.retail_price) < 0.25 THEN '2_Medium_Discount_10_25'
                ELSE '3_High_Discount_25_Plus'
            END AS discount_tier

        FROM
            `bigquery-public-data.thelook_ecommerce.orders` AS t1
        INNER JOIN
            `bigquery-public-data.thelook_ecommerce.order_items` AS t2
            ON t1.order_id = t2.order_id
        INNER JOIN
            `bigquery-public-data.thelook_ecommerce.products` AS t3
            ON t2.product_id = t3.id
        INNER JOIN
            `bigquery-public-data.thelook_ecommerce.users` AS t4
            ON t1.user_id = t4.id
        WHERE
            t1.status IN ('Complete', 'Shipped')
            AND t3.category = '{TARGET_CATEGORY}'
            AND t1.user_id IN (SELECT user_id FROM repeat_users)
            AND t2.sale_price <= t3.retail_price
    )

-- Final SELECT: Aggregate metrics by the chosen drivers
SELECT
    -- Grouping Drivers
    country,
    traffic_source, -- CORRECTION: Swapped in traffic_source
    discount_tier,

    -- Metrics
    COUNT(DISTINCT order_id) AS total_orders,
    SUM(sale_price) AS total_revenue,
    AVG(sale_price) AS average_order_value,
    AVG(discount_pct) AS average_discount_applied

FROM
    target_transactions
GROUP BY
    1, 2, 3
ORDER BY
    total_revenue DESC
LIMIT 20
"""

# Execute the corrected query
print(f"Executing BigQuery query for drivers analysis on '{TARGET_CATEGORY}' purchases by Repeat Customers...")

try:
    # Run the query and convert to a Pandas DataFrame
    driver_analysis_df = client.query(QUERY_DRIVERS_CORRECTED).to_dataframe()

    # --- Presentation ---
    pd.options.display.float_format = '{:,.2f}'.format

    print("\n--- Driver Analysis: Repeat Customer Purchases of Jeans (FINAL CORRECTION: Using Traffic Source) ---")
    print(f"Metrics aggregated by Country, Traffic Source, and Discount Tier (N={len(driver_analysis_df)} rows)")

    # Format discount percentage for display
    driver_analysis_df['average_discount_applied'] = pd.to_numeric(driver_analysis_df['average_discount_applied'], errors='coerce')
    driver_analysis_df['average_discount_applied'] = driver_analysis_df['average_discount_applied'].apply(lambda x: f"{x:.2%}")

    print(driver_analysis_df)

except NameError:
    print("Cannot run query: BigQuery client was not successfully initialized.")
except Exception as e:
    print(f"An error occurred during query execution: {e}")



Executing BigQuery query for drivers analysis on 'Jeans' purchases by Repeat Customers...

--- Driver Analysis: Repeat Customer Purchases of Jeans (FINAL CORRECTION: Using Traffic Source) ---
Metrics aggregated by Country, Traffic Source, and Discount Tier (N=20 rows)
           country traffic_source  discount_tier  total_orders  total_revenue  \
0            China         Search  0_No_Discount           674      70,408.67   
1    United States         Search  0_No_Discount           435      42,994.07   
2           Brasil         Search  0_No_Discount           279      28,772.73   
3            China        Organic  0_No_Discount           137      14,036.31   
4      South Korea         Search  0_No_Discount           107      10,475.94   
5    United States        Organic  0_No_Discount            95       9,642.62   
6   United Kingdom         Search  0_No_Discount            91       8,886.34   
7          Germany         Search  0_No_Discount            88       8,067.08   
8 

### Summary of Key Findings

The data reveals an overwhelming pattern: Repeat customers purchasing Jeans are highly resilient to price changes and are primarily driven by search, concentrated in a few key international markets.

Discount Effectiveness:
* The most striking observation is that every single entry in the top 20 revenue rows is categorized as 0_No_Discount.
* Conclusion: For Repeat Customers buying Jeans, discounting is unnecessary and would only erode profit margins. This customer segment is making a mission-based purchase, driven by brand loyalty or necessity, not price.

Geographic Concentration:
* China (Revenue Leader): The top segment (China / Search / No Discount) alone generated over $70k in revenue, nearly double the next closest segment (United States).
* Tier 1 Markets: China, United States, and Brasil dominate the revenue list. These three countries represent critical markets for this product.

Channel Performance (Traffic Source):
* Search Power: Search is overwhelmingly the most effective traffic source for generating revenue from these valuable customers.
* Organic Value: Organic traffic is the second most valuable source (e.g., China Organic is the 4th highest segment), indicating strong historical SEO performance or direct navigation following initial discovery.
* Social/Display: While channels like Facebook, Display, and Email appear lower on the list, they still drive substantial revenue without discounts, showing that multi-channel engagement works, but the initial search intent is highly correlated with the final high-value purchase.



## **VALIDATE:**

### Prompt 5: Validate

> Lets cross-check all three of these insights with alternative queries or counterexamples.
Lets look for a case where the first answer was misleading and how we can correct it.



### Cross-Check 1: Discount Sensitivity (The Counter-Example)

Initial Insight: Discounts are unnecessary for Repeat Customers buying Jeans (Top 20 rows showed 0% discount).

Challenge: Does this hold true for a lower-performing or more competitive product category? If discounts are a major driver for a different, potentially slower-moving category, then the insight is specific to Jeans, not a company-wide truth.

Query Goal: Analyze the purchasing drivers for a new category, T-shirts, focusing only on discounted purchases (Discount Tier 3 or higher).

In [12]:
# --- Cross-Check 1.1: Discount Sensitivity for ALL Products (Repeat Buyers) ---

QUERY_CROSS_CHECK_DISCOUNT_ALL = f"""
WITH
    -- CTE 1: Identify Repeat Customers
    repeat_users AS (
        SELECT
            user_id
        FROM
            `bigquery-public-data.thelook_ecommerce.orders`
        WHERE
            status IN ('Complete', 'Shipped')
        GROUP BY
            user_id
        HAVING
            COUNT(order_id) >= {MIN_ORDERS_FOR_REPEAT}
    ),

    -- CTE 2: Calculate discount metrics for ALL products purchased by Repeat Customers
    target_transactions AS (
        SELECT
            t1.order_id,
            t2.sale_price,
            t3.retail_price,

            -- Calculate discount percentage
            SAFE_DIVIDE(
                (t3.retail_price - t2.sale_price),
                t3.retail_price
            ) AS discount_pct,

            -- Create Discount Tiers for grouping
            CASE
                WHEN (t3.retail_price - t2.sale_price) <= 0 THEN '0_No_Discount'
                WHEN SAFE_DIVIDE((t3.retail_price - t2.sale_price), t3.retail_price) < 0.10 THEN '1_Low_Discount_1_10'
                WHEN SAFE_DIVIDE((t3.retail_price - t2.sale_price), t3.retail_price) < 0.25 THEN '2_Medium_Discount_10_25'
                ELSE '3_High_Discount_25_Plus'
            END AS discount_tier

        FROM
            `bigquery-public-data.thelook_ecommerce.orders` AS t1
        INNER JOIN
            `bigquery-public-data.thelook_ecommerce.order_items` AS t2
            ON t1.order_id = t2.order_id
        INNER JOIN
            `bigquery-public-data.thelook_ecommerce.products` AS t3
            ON t2.product_id = t3.id
        WHERE
            t1.status IN ('Complete', 'Shipped')
            AND t1.user_id IN (SELECT user_id FROM repeat_users)
            AND t2.sale_price <= t3.retail_price
    )

-- Final SELECT: Compare revenue distribution based on discount tier for all products
SELECT
    discount_tier,
    COUNT(order_id) AS total_orders,
    SUM(sale_price) AS total_revenue,
    AVG(sale_price) AS average_order_value,
    AVG(discount_pct) AS average_discount_applied
FROM
    target_transactions
GROUP BY
    discount_tier
ORDER BY
    total_revenue DESC
"""

# Execute the query
print(f"\n--- Cross-Check 1.1: Discount Sensitivity for Repeat Buyers (ALL Products) ---")
try:
    df_discount_all_check = client.query(QUERY_CROSS_CHECK_DISCOUNT_ALL).to_dataframe()
    df_discount_all_check['average_discount_applied'] = pd.to_numeric(df_discount_all_check['average_discount_applied'], errors='coerce')
    df_discount_all_check['average_discount_applied'] = df_discount_all_check['average_discount_applied'].apply(lambda x: f"{x:.2%}")
    print(df_discount_all_check)
except Exception as e:
    print(f"Error executing Cross-Check 1.1: {e}")



--- Cross-Check 1.1: Discount Sensitivity for Repeat Buyers (ALL Products) ---
   discount_tier  total_orders  total_revenue  average_order_value  \
0  0_No_Discount         41399   2,465,883.80                59.56   

  average_discount_applied  
0                    0.00%  


Initial Insight: Discounts are unnecessary for Repeat Customers buying Jeans.

Conclusion on Discounting: Insight Confirmed and Amplified.

The initial finding was not misleading—it was just incomplete. The data shows that 100% of the revenue generated by the Repeat Customer segment came from transactions where no discount was applied.

### Cross-Check 2: Geographic Dominance (The Counter-Segment)

Initial Insight: China is the dominant revenue market for Repeat Customers purchasing Jeans.

Challenge: Is China the dominant market for all customers, or just the high-value repeat segment? If the US or another market dominates the New Customer segment, our overall growth strategy might be too narrow.

Query Goal: Analyze the revenue distribution by country for First-Time Buyers (New Customers) across all product categories.

In [10]:
# Define the segment for the counter-check
MIN_ORDERS_FOR_NEW_BUYER = 1

QUERY_CROSS_CHECK_GEOGRAPHY = f"""
WITH
    -- CTE 1: Identify First-Time Buyers (Users with EXACTLY 1 successful order)
    first_time_users AS (
        SELECT
            user_id
        FROM
            `bigquery-public-data.thelook_ecommerce.orders`
        WHERE
            status IN ('Complete', 'Shipped')
        GROUP BY
            user_id
        HAVING
            COUNT(order_id) = {MIN_ORDERS_FOR_NEW_BUYER}
    ),

    -- CTE 2: Calculate revenue for first-time buyers (across all products)
    first_time_transactions AS (
        SELECT
            t1.order_id,
            t2.sale_price,
            t3.country
        FROM
            `bigquery-public-data.thelook_ecommerce.orders` AS t1
        INNER JOIN
            `bigquery-public-data.thelook_ecommerce.order_items` AS t2
            ON t1.order_id = t2.order_id
        INNER JOIN
            `bigquery-public-data.thelook_ecommerce.users` AS t3
            ON t1.user_id = t3.id
        WHERE
            t1.user_id IN (SELECT user_id FROM first_time_users)
    )

-- Final SELECT: Aggregate revenue by country for New Customers
SELECT
    country,
    COUNT(DISTINCT order_id) AS total_orders,
    SUM(sale_price) AS total_revenue
FROM
    first_time_transactions
GROUP BY
    country
ORDER BY
    total_revenue DESC
LIMIT 10
"""

# Execute the query
print(f"\n--- Cross-Check 2: Geographic Dominance for First-Time Buyers (All Categories) ---")
try:
    df_geo_check = client.query(QUERY_CROSS_CHECK_GEOGRAPHY).to_dataframe()
    print(df_geo_check)
except Exception as e:
    print(f"Error executing Cross-Check 2: {e}")



--- Cross-Check 2: Geographic Dominance for First-Time Buyers (All Categories) ---
          country  total_orders  total_revenue
0           China         18889   1,626,271.65
1   United States         12590   1,089,789.17
2          Brasil          8242     701,995.33
3     South Korea          3201     271,691.20
4          France          2601     225,818.07
5  United Kingdom          2634     225,635.27
6           Spain          2272     204,036.43
7         Germany          2328     202,772.19
8           Japan          1361     118,040.61
9       Australia          1286     112,586.67


Analysis of Cross-Check 2 Output

Initial Insight (Jeans/Repeat Customers): China is the dominant revenue market.

Conclusion on Geographic Dominance: Insight Confirmed.

The pattern of geographic dominance holds true across the entire customer base, regardless of customer segment (Repeat vs. First-Time) or product category (Jeans vs. All).

China is definitively the largest market, followed by the United States, and then Brasil.
This suggests that the business model, marketing reach, or product market fit is strongest in these three regions.

### Cross-Check 3: Traffic Source Strength (The True AOV Test)

Initial Insight: Search is the dominant high-revenue source for Repeat Jeans buyers.

Challenge: Does Search also deliver the highest Average Order Value (AOV), or are there niche channels (like Email or Display) that bring in fewer, but much more valuable, transactions?

Query Goal: Rank all traffic sources purely by the highest AOV across all successful transactions, regardless of customer type or product.

In [11]:
QUERY_CROSS_CHECK_AOV = """
WITH
    -- CTE 1: Calculate Total Revenue and Orders per Order ID
    order_totals AS (
        SELECT
            order_id,
            SUM(sale_price) AS order_revenue
        FROM
            `bigquery-public-data.thelook_ecommerce.order_items`
        GROUP BY
            order_id
    )

-- Final SELECT: Join orders/users to calculate AOV per traffic source
SELECT
    t2.traffic_source,
    COUNT(t1.order_id) AS total_orders,
    SUM(t3.order_revenue) AS total_revenue,
    AVG(t3.order_revenue) AS average_order_value -- AOV
FROM
    `bigquery-public-data.thelook_ecommerce.orders` AS t1
INNER JOIN
    `bigquery-public-data.thelook_ecommerce.users` AS t2
    ON t1.user_id = t2.id
INNER JOIN
    order_totals AS t3
    ON t1.order_id = t3.order_id
WHERE
    t1.status IN ('Complete', 'Shipped')
GROUP BY
    t2.traffic_source
HAVING
    COUNT(t1.order_id) > 100 -- Ignore sources with negligible volume
ORDER BY
    average_order_value DESC
"""

# Execute the query
print(f"\n--- Cross-Check 3: Traffic Source Ranking by Overall AOV ---")
try:
    df_aov_check = client.query(QUERY_CROSS_CHECK_AOV).to_dataframe()
    print(df_aov_check)
except Exception as e:
    print(f"Error executing Cross-Check 3: {e}")




--- Cross-Check 3: Traffic Source Ranking by Overall AOV ---
  traffic_source  total_orders  total_revenue  average_order_value
0        Display          2823     252,267.67                89.36
1         Search         48278   4,157,657.58                86.12
2          Email          3417     294,042.54                86.05
3        Organic         10092     864,618.57                85.67
4       Facebook          4032     336,635.81                83.49


### Analysis of Cross-Check 3 Output:

Initial Insight (Jeans/Repeat Customers): Search is the dominant high-revenue source.

Conclusion on Traffic Source: Insight Challenged and Corrected.

While Search remains the overwhelming volume and revenue driver (48k orders, $4.1M revenue), it does not deliver the highest Average Order Value.

**Correction: The Display channel generates the highest AOV ($89.36).**

Implication: Display customers, while fewer in number, tend to buy more expensive items or bundle more items in their cart. This suggests that the visual, brand-focused nature of Display advertising leads to higher-quality transactions.

## **EXTEND:**

### Pompt 6: Visualize

>

    Build one interactive Plotly chart in Colab with:

        Scorecard: revenue (or profit), last 30 days

        Pie/Donut: sales % by region or channel

        Bar: top 5 products/categories


In [14]:
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from datetime import datetime, timedelta

# Initialize BigQuery Client (assuming this is done)
try:
    client = bigquery.Client()
except Exception as e:
    print(f"Error initializing BigQuery client: {e}")
    client = None

if client is None:
    print("Cannot proceed without BigQuery client initialization.")
else:
    # --- 1. Scorecard Data: Revenue Last 30 Days ---

    # Determine the end date (latest date in the dataset)
    MAX_DATE_QUERY = """
    SELECT
        MAX(DATE(created_at))
    FROM
        `bigquery-public-data.thelook_ecommerce.orders`
    WHERE
        status IN ('Complete', 'Shipped')
    """
    max_date_result = client.query(MAX_DATE_QUERY).to_dataframe()
    max_date = max_date_result.iloc[0, 0] if not max_date_result.empty else datetime.now().date()

    end_date = max_date
    start_date = end_date - timedelta(days=30)

    SCORECARD_QUERY = f"""
    SELECT
        SUM(t2.sale_price) AS total_revenue
    FROM
        `bigquery-public-data.thelook_ecommerce.orders` AS t1
    INNER JOIN
        `bigquery-public-data.thelook_ecommerce.order_items` AS t2
        ON t1.order_id = t2.order_id
    WHERE
        t1.status IN ('Complete', 'Shipped')
        AND DATE(t1.created_at) >= '{start_date}'
        AND DATE(t1.created_at) <= '{end_date}'
    """
    df_scorecard = client.query(SCORECARD_QUERY).to_dataframe()
    last_30_day_revenue = df_scorecard['total_revenue'].iloc[0] if not df_scorecard.empty else 0

    # --- 2. Pie Chart Data: Sales % by Traffic Source (Channel) ---
    # Aliases: orders=t1, users=t2, order_items=t3
    PIE_QUERY = """
    SELECT
        t2.traffic_source,
        SUM(t3.sale_price) AS total_revenue
    FROM
        `bigquery-public-data.thelook_ecommerce.orders` AS t1
    INNER JOIN
        `bigquery-public-data.thelook_ecommerce.users` AS t2
        ON t1.user_id = t2.id
    INNER JOIN
        `bigquery-public-data.thelook_ecommerce.order_items` AS t3
        ON t1.order_id = t3.order_id
    WHERE
        t1.status IN ('Complete', 'Shipped')
    GROUP BY
        t2.traffic_source
    HAVING
        total_revenue > 0
    ORDER BY
        total_revenue DESC
    """
    df_pie = client.query(PIE_QUERY).to_dataframe()

    # --- 3. Bar Chart Data: Top 5 Categories ---
    # Aliases: orders=t1, order_items=t2, products=t3 (Order fixed here)
    BAR_QUERY = """
    SELECT
        t3.category,
        SUM(t2.sale_price) AS total_revenue
    FROM
        `bigquery-public-data.thelook_ecommerce.orders` AS t1
    INNER JOIN
        `bigquery-public-data.thelook_ecommerce.order_items` AS t2
        ON t1.order_id = t2.order_id
    INNER JOIN
        `bigquery-public-data.thelook_ecommerce.products` AS t3
        ON t2.product_id = t3.id
    WHERE
        t1.status IN ('Complete', 'Shipped')
    GROUP BY
        t3.category
    ORDER BY
        total_revenue DESC
    LIMIT 5
    """
    df_bar = client.query(BAR_QUERY).to_dataframe()


    # =========================================================================
    # PLOTLY Visualization
    # =========================================================================

    # Create subplot structure: 2 rows, 2 columns.
    fig = make_subplots(
        rows=2, cols=2,
        specs=[
            [{"type": "indicator"}, {"type": "domain"}],
            [{"type": "bar", "colspan": 2}, None]
        ],
        row_heights=[0.5, 0.5],
        column_widths=[0.5, 0.5],
        subplot_titles=(
            f"Revenue Last 30 Days ({start_date} to {end_date})",
            "Revenue Share by Traffic Channel",
            "Top 5 Product Categories by Total Revenue"
        )
    )

    # --- 1. Scorecard (Indicator) ---
    fig.add_trace(
        go.Indicator(
            mode="number",
            value=last_30_day_revenue,
            number={'prefix': "$", 'valueformat': ",.0f"},
            title={'text': "Total Revenue"},
        ),
        row=1, col=1
    )

    # --- 2. Pie Chart (Sales % by Channel) ---
    fig.add_trace(
        go.Pie(
            labels=df_pie['traffic_source'],
            values=df_pie['total_revenue'],
            hole=.4,
            marker_colors=['#003f5c', '#58508d', '#bc5090', '#ff6361', '#ff843a']
        ),
        row=1, col=2
    )

    # --- 3. Bar Chart (Top 5 Categories) ---
    fig.add_trace(
        go.Bar(
            x=df_bar['category'],
            y=df_bar['total_revenue'],
            text=df_bar['total_revenue'].apply(lambda x: f'${x:,.0f}'),
            textposition='auto',
            marker_color='#ff6361'
        ),
        row=2, col=1
    )

    # Update Layout
    fig.update_layout(
        title_text="E-commerce Growth Dashboard (Key Metrics & Drivers)",
        height=700,
        showlegend=True,
        # Customize Bar Chart Y-axis
        yaxis2={'title': 'Total Revenue ($)'}
    )

    # Hide the second column's y-axis for the bar chart since it spans both columns
    fig.update_xaxes(showticklabels=True, row=2, col=1)

    # Show the interactive figure
    fig.show()


## Strategic Recommendations
### **Recommendation 1:** Profit Optimization through Targeted Discount Elimination (Focus on Repeat Customers)

**Strategy:** Segment the CRM and marketing automation platforms to ensure that all targeted promotional communications (email, SMS, push notifications) directed at users categorized as "Repeat Customers" (those with $\ge 2$ completed orders) exclusively promote non-monetary value adds (e.g., free premium shipping, early access to new lines, exclusive loyalty events) rather than percentage discounts or dollar-off coupons.

**Specific Action:** Audit the top-performing traffic_source (Search) campaigns targeting this segment (China, US, Brazil) and replace any price-based ads with messaging emphasizing product authority and speed/convenience. This action immediately converts marketing budget previously spent on unnecessary discounts into pure profit margin without sacrificing volume in the highest-value segment.

### **Recommendation 2:** Scale High-AOV Behavior by Mimicking the Display Channel

**Strategy:** Increase the overall Average Order Value (AOV) across the highest-volume channel (Search) by systematically identifying and replicating the product bundling, visual style, and item focus that drive high AOV in the Display channel.

**Specific Action:** Work with the performance marketing team to analyze the creative assets and landing pages used in the Display channel (which generates the highest AOV of $89.36). Extract the top-selling product combinations (e.g., if Display is promoting a 'Shirt + Pants' bundle). Use this knowledge to implement dynamic product bundling and cross-sell promotion modules directly into the shopping cart and landing pages associated with high-traffic Search campaigns. This transfers the higher-AOV behavior seen in Display to the large transaction volume of Search.
